#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [6]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [7]:
df = pd.read_csv('data/StudentsPerformance.csv')

In [8]:
df.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Preparing X and Y variables

In [9]:
X = df.drop(columns=['math_score'],axis=1)

In [10]:
X.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [26]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [12]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race/ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [13]:
y = df['math_score']

In [14]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [27]:
# Assuming df is your original DataFrame
X = df.drop(columns=['math_score'], axis=1)  # X is a DataFrame
y = df['math_score']  # Target variable

# Select numeric and categorical features
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Define transformers
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

# Create ColumnTransformer
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

print("Numeric Features:", num_features)
print("Categorical Features:", cat_features)


Numeric Features: Index(['reading_score', 'writing_score'], dtype='object')
Categorical Features: Index(['gender', 'race/ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')


In [28]:
X = preprocessor.fit_transform(X)

In [29]:
X.shape

(1000, 19)

In [30]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

#### Create an Evaluate Function to give all metrics after model Training

In [31]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

# Training and evaluation
for model_name, model in models.items():
    model.fit(X_train, y_train)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate
    train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

    model_list.append(model_name)
    r2_list.append(test_r2)

    print(f"{model_name}")
    print("Training set:")
    print(f"- RMSE: {train_rmse:.4f}, MAE: {train_mae:.4f}, R2: {train_r2:.4f}")
    print("Test set:")
    print(f"- RMSE: {test_rmse:.4f}, MAE: {test_mae:.4f}, R2: {test_r2:.4f}")
    print("=" * 30)

# Save results in DataFrame
results_df = pd.DataFrame({"Model": model_list, "Test R2 Score": r2_list})
print(results_df.sort_values(by="Test R2 Score", ascending=False))

Linear Regression
Training set:
- RMSE: 5.3231, MAE: 4.2667, R2: 0.8743
Test set:
- RMSE: 5.3940, MAE: 4.2148, R2: 0.8804
Lasso
Training set:
- RMSE: 6.5938, MAE: 5.2063, R2: 0.8071
Test set:
- RMSE: 6.5197, MAE: 5.1579, R2: 0.8253
Ridge
Training set:
- RMSE: 5.3233, MAE: 4.2650, R2: 0.8743
Test set:
- RMSE: 5.3904, MAE: 4.2111, R2: 0.8806
K-Neighbors Regressor
Training set:
- RMSE: 5.7077, MAE: 4.5167, R2: 0.8555
Test set:
- RMSE: 7.2530, MAE: 5.6210, R2: 0.7838
Decision Tree
Training set:
- RMSE: 0.2795, MAE: 0.0187, R2: 0.9997
Test set:
- RMSE: 7.5200, MAE: 6.0800, R2: 0.7676
Random Forest Regressor
Training set:
- RMSE: 2.2627, MAE: 1.8111, R2: 0.9773
Test set:
- RMSE: 5.9094, MAE: 4.5730, R2: 0.8565
XGBRegressor
Training set:
- RMSE: 1.0073, MAE: 0.6875, R2: 0.9955
Test set:
- RMSE: 6.4733, MAE: 5.0577, R2: 0.8278
CatBoosting Regressor
Training set:
- RMSE: 3.0427, MAE: 2.4054, R2: 0.9589
Test set:
- RMSE: 6.0086, MAE: 4.6125, R2: 0.8516
AdaBoost Regressor
Training set:
- RMSE: 5.